In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import cv2
from google.colab.patches import cv2_imshow

In [3]:
def map_char(char):
  dict_char = {
      '0':0,'1':1,'2':2,'3':3,'4':4,'5':5,'6':6,'7':7,'8':8,'9':9,
      'A':10,  'B':11, 'C':12, 'D':13, 'E':14, 'F':15, 'G':16, 'H':17, 'I':18 ,'J':19, 'K':20, 
      'L':21, 'M':22, 'N':23, 'O':24, 'P':25, 'Q':26, 'R':27, 'S':28, 'T':29, 'U':30, 
      'V':31, 'W':32, 'X':33, 'Y':34, 'Z':35
  }
  return dict_char[char]

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
def find_contours(dimensions, img) :

    # Find all contours in the image
    cntrs, _ = cv2.findContours(img.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    # Retrieve potential dimensions
    lower_width = dimensions[0]
    upper_width = dimensions[1]
    lower_height = dimensions[2]
    upper_height = dimensions[3]
    
    # Check largest 5 or  15 contours for license plate or character respectively
    cntrs = sorted(cntrs, key=cv2.contourArea, reverse=True)[:15]
    
    ii = cv2.imread('contour.jpg')
    
    x_cntr_list = []
    target_contours = []
    img_res = []
    for cntr in cntrs :
        # detects contour in binary image and returns the coordinates of rectangle enclosing it
        intX, intY, intWidth, intHeight = cv2.boundingRect(cntr)
        
        # checking the dimensions of the contour to filter out the characters by contour's size
        if intWidth > lower_width and intWidth < upper_width and intHeight > lower_height and intHeight < upper_height :
            x_cntr_list.append(intX) #stores the x coordinate of the character's contour, to used later for indexing the contours

            char_copy = np.zeros((44,24))
            # extracting each character using the enclosing rectangle's coordinates.
            char = img[intY:intY+intHeight, intX:intX+intWidth]
            char = cv2.resize(char, (20, 40))
            
            cv2.rectangle(ii, (intX,intY), (intWidth+intX, intY+intHeight), (50,21,200), 2)
            plt.imshow(ii, cmap='gray')

            # Make result formatted for classification: invert colors
            char = cv2.subtract(255, char)

            # Resize the image to 24x44 with black border
            char_copy[2:42, 2:22] = char
            char_copy[0:2, :] = 0
            char_copy[:, 0:2] = 0
            char_copy[42:44, :] = 0
            char_copy[:, 22:24] = 0

            img_res.append(char_copy) # List that stores the character's binary image (unsorted)
            
    # Return characters on ascending order with respect to the x-coordinate (most-left character first)
            
    plt.show()
    # arbitrary function that stores sorted list of character indeces
    indices = sorted(range(len(x_cntr_list)), key=lambda k: x_cntr_list[k])
    img_res_copy = []
    for idx in indices:
        img_res_copy.append(img_res[idx])# stores character images according to their index
    img_res = np.array(img_res_copy)

    return img_res

In [5]:
def segment_characters(image) :

    # Preprocess cropped license plate image
    img_lp = cv2.resize(image, (333, 75))
    img_gray_lp = cv2.cvtColor(img_lp, cv2.COLOR_BGR2GRAY)
    _, img_binary_lp = cv2.threshold(img_gray_lp, 200, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
    img_binary_lp = cv2.erode(img_binary_lp, (3,3))
    img_binary_lp = cv2.dilate(img_binary_lp, (3,3))

    LP_WIDTH = img_binary_lp.shape[0]
    LP_HEIGHT = img_binary_lp.shape[1]

    # Make borders white
    img_binary_lp[0:3,:] = 255
    img_binary_lp[:,0:3] = 255
    img_binary_lp[72:75,:] = 255
    img_binary_lp[:,330:333] = 255

    # Estimations of character contours sizes of cropped license plates
    dimensions = [LP_WIDTH/6,
                       LP_WIDTH/2,
                       LP_HEIGHT/10,
                       2*LP_HEIGHT/3]
    plt.imshow(img_binary_lp, cmap='gray')
    plt.show()
    cv2.imwrite('contour.jpg',img_binary_lp)

    # Get contours within cropped license plate
    char_list = find_contours(dimensions, img_binary_lp)

    return char_list

In [7]:
# with open(path_lbl) as f:
#     lines = f.readlines()
#     print(lines[0])

In [101]:
path_cnn = '/content/drive/My Drive/Data/bien-so-xe/cnn'
try:
  os.mkdir(path_cnn)
except:
  print('exists folder')


for f in ['train','val']:
  path = os.path.join(path_cnn, str(f))
  try:
    os.mkdir(path)
  except:
    print('exists folder')
  for i in range(36):
    p = os.path.join(path, str(i))
    try:
      os.mkdir(p)
    except:
      print('exists folder')



In [14]:
path_img='/content/drive/My Drive/Data/bien-so-xe/img_cnn/train/ML13.jpg'
path_lbl='/content/drive/My Drive/Data/bien-so-xe/lbl_cnn/train/ML13.txt'
path_folder_save = '/content/drive/My Drive/Data/bien-so-xe/cnn/train'

def split_save_data(path_img,path_lbl,path_folder_save):
  # try:
  print('path_img',path_img)
  img = cv2.imread(path_img)
  char = segment_characters(img)
  with open(path_lbl) as f:
      lines = f.readlines()
  label = lines[0]
  if(len(char)>=9 and len(char) == len(label)):
    for i in range(len(char)):
        class_ = map_char(str(label[i]))
        path = os.path.join(path_folder_save, str(class_))
        filename = str(uuid.uuid4())+'=='
        filename+=path_img.split('/')[-1]
        path = os.path.join(path, f'{filename}.jpg')
        # cv2_imshow(char[i])
        print(path)
        cv2.imwrite(path, char[i])
  # except:
  #   print('error', path_img)

  return 0

# split_save_data(path_img,path_lbl,path_folder_save)

In [16]:
import uuid
import os
path_folder = '/content/drive/My Drive/Data/bien-so-xe/img_cnn/val'
path_folder_save = '/content/drive/My Drive/Data/bien-so-xe/cnn/val'

for dirname, _, filenames in os.walk(path_folder):
    for filename in filenames:
      if('.DS_Store' not in filenames):
        path_img = os.path.join(dirname, filename)
        duoi_anh = path_img.split('.')[-1]

        path_lbl = path_img.replace('/img_cnn','/lbl_cnn')
        path_lbl = path_lbl.replace(duoi_anh,'txt')
        split_save_data(path_img,path_lbl,path_folder_save)

        
     

      

Output hidden; open in https://colab.research.google.com to view.

In [85]:
label = 'ML05F9854'
label[8]

'4'